In [74]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
import re
import sys
from tqdm import tqdm
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, TfidfVectorizer
from sklearn.cluster import KMeans
import pymongo

In [2]:
sys.path.insert(1, '../../')
import util

In [7]:
url = "https://www.indeed.co.in/jobs?q=civil+engineering&l=India"

In [8]:
r = requests.get(url)
p = bs(r.text)

In [9]:
def totextfile(filename, title, skills, des):
    exists = os.path.isfile("./" + filename + ".txt")
    if exists:
        print("Job present in directory.")
        return      
    else:
        file = filename + '.txt'
        file1 = open(file,'wb')
        file1.write(title.encode('utf8'))
        file1.write("\n".encode('utf8'))
        for i in skills:
            file1.write(i.encode('utf8'))
            file1.write("\n".encode('utf8'))
        for i in des:
            file1.write(i.encode('utf8'))
            file1.write("\n".encode('utf8'))
        file1.close()
        print(filename, " Done")


In [18]:
def jobtextadd(pag, count, titles):
    for i in pag:
        c = 'https://www.indeed.co.in'+i['href']
        r2 = requests.get(c)
        p1 = bs(r2.text)
        page = p1.find_all('a', {'class':'jobtitle'})
        for i in page:
            c = 'https://www.indeed.co.in' + i['href']
            r1 = requests.get(c)
            r1.url
            page1 = bs(r1.text)
            tit = page1.find('div', {'class' : 'jobsearch-JobInfoHeader-title-container'})
            name = page1.find('div', {'class' : 'jobsearch-CompanyInfoWithoutHeaderImage'})
            title = tit.text
            filename = title + ' ' + name.text
            filename = filename.replace('/','').replace('|','').replace('""','')
            if filename in titles:
                print("Job already present.\n")
                continue
            else:
                skill = page1.find_all('span',{'class' : 'jobsearch-JobMetadataHeader-skillItem'})
                description = page1.find_all('div', {'class' : 'jobsearch-jobDescriptionText'})
                skills = []
                des = []
                for j in skill:
                    skills.append(j.text)
                for k in description:
                    des.append(k.text)
                totextfile(filename,title,skills,des)
                count += 1
                #print(count)
        pages = p1.find_all('div',{'class':'pagination'})
        pag1 = pages[0].find_all('a')
        for p in pag1:
            if p in pag:
                continue
            else:
                pag.append(p)
        if count >= 100:
            print("Stopping!")
            break
        print (i," done")
        

In [19]:
pages = p.find_all('div',{'class':'pagination'})
pag = set(pages[0].find_all('a'))
pag = list(pag)
count = 0
titles = []
jobtextadd(pag,count,titles) 

Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
Job present in directory.
<a class="jobtitle turnstileLink" data-tn-element="jobTitle" href="/pagead/clk?mo=r&amp;ad=-6NYlbfkN0AyDUXvIbtIMLsYoak0xyD2czxZai7hkMhNco0btN9NWrj59XrNQ9gMF9yTotYlw-J81h0S7Ivq2zPXHIlYEiQnkz8GZCNQUVmWhiX490G3OTkaH4HH_Qu0rlqj4uIC1npbd55fVpJCRkgh7k1F9vkG9JVrX-e5JlzyLx_bg69HR97FDwNt9jWXpCmRJRZsZIaXBd4VYYvvDwViDbSX8Nd5opMj76eh2MZwf-5cbJPpWPsa8rDxyneKw_VxB3KVQzileIHrCYRlal1d1S2bq5xWyqKhdnlUVwivuvOs8buAtUJcbsVkBOA5Wz43a8dlvdfuY2vpcYpbafhNjEStIeEIJuhAIBtr2y1AqaDwsx9VIEE3gWe0X3UiTJMIsbQ2B2XCy1FJaAH4SOJDYA48ApJyDstkki86EeciBmPxA6qPoBqxHjwN9bJN8vMWz03WlFpOGF6P37vWUA==&amp;p=15&amp;fvj=1&a

## Making job database

In [3]:
files= os.listdir('./')
html_path = "./tempHtml"

In [82]:
jobs = []
for f in tqdm(files):
    if f.split('.')[-1] not in ('txt', 'pdf', 'docx', 'json'):
        continue
    job = {}
    job['name'] = f
    job['path'] = './data/Test Jobs'
    j = util.extract_job_keywords(f, '../CORPUS.json', html_path)
    final = {}
    for x,y in j.items():
        for z in y:
            final[z.replace('.', '_')] = 1
    job.update(final)
    jobs.append(job)

100%|██████████| 1068/1068 [00:08<00:00, 124.11it/s]


In [83]:
train_data = pd.DataFrame(jobs).fillna(0).to_dict('records')

In [84]:
Client = pymongo.MongoClient('mongodb://localhost:27017')

In [85]:
db = Client.users

In [86]:
db.jobs.insert_many(train_data)

In [87]:
db.jobs.count()

1060

In [64]:
km = KMeans(max_iter=100, n_clusters=5)

In [65]:
km.fit(train_data.values[:, 1:])

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
       n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [71]:
labels = km.labels_
cluster = {}
for i in range(len(labels)):
    try:
        cluster[labels[i]].append(jobs[i]['name'])
    except:
        cluster[labels[i]] = [jobs[i]['name']]

In [72]:
cluster

{2: ['HR, Admin & Compliance Executive BRITANNIA GARMENT PACKAGING.txt',
  'Manager, Customer Service CommScope459 reviews.txt',
  'Relationship Manager Yes Bank819 reviews.txt',
  'Manager Operations & Customer Service Box Office Logistics.txt',
  'Business Analyst Senior (G2) TSYS1,400 reviews.txt',
  'Manager - Customer Service TATA Capital175 reviews.txt',
  'Business Development Representative CallE Corp.txt',
  'Customer Relations Manager BUILDTECH BUILDERS.txt',
  'Finance & Accounts Executive Ritam Steels Pvt. Ltd..txt',
  'Senior Manager – Finance and Accounts Bayleaf Hr Solutions Pvt Ltd.txt',
  'Finance & Accounts Manager I curve careers.txt',
  'Interior Site Supervisor Mobius interior concepts pvt ltd.txt',
  'Technology Team Manager Telstra1,242 reviews.txt',
  'Business Development Executive  Manager Simform Solutions Pvt. Ltd..txt',
  'Manager Finance & Accounts Kamdhenu Steel2 reviews.txt',
  'Assistant ManagerManager - Accounts & Finance HR Planners and Developers Pvt